
# Scrapping de los Investigadores de la UIB
## Jun De Wu

El objetivo del trabajo es entrar en la página web de la UIB, en concreto en la sección de grupos de investigación. Dentro de esta página, recorrer por cada grupo extrayendo información de los investigadores como grupo de investigación, nombre, género, titulación, puesto y currículum vitae. 

Empezamos por cargar las librerías que vamos a necesitar.

In [1]:
from bs4 import BeautifulSoup
import requests
import json

Definimos dos funciones que sirven para grabar y leer ficheros para no tener que volver a realizar todo el proceso cada vez que entramos a modificar algo del trabajo.

In [2]:
def dump_data(data, filename):
    with open(filename, "w") as out_file:
        json.dump(data, out_file)
        
def load_data(filename):
    data = None
    with open(filename, "r") as in_file:
        data = json.load(in_file)
    return data

Antes de entrar de lleno a explicar las funciones que emplearemos a lo largo del documento, vamos a explicar la idea general del trabajo. Nosotros empezamos por acceder a las páginas de la UIB donde están numeradas los grupos de investigación en 15 páginas diferentes. El enlace para la página número 1 es https://www.uib.cat/recerca/estructures/grups/grups_area/id_area=-1%2526npag=1 y podemos acceder a las otra cambiando el último 1 por la página que queremos acceder. Dentro de cada grupo tenemos que acceder al equipo de investigación y aquí es donde viene el problema. De forma general, tomando como ejemplo la página web del grupo "Gènetica Humana", las páginas web de los grupos vienen dado de la forma https://www.uib.cat/recerca/estructures/grups/grup/GENEHUMA/ + "equip/index.html". 

Ahora bien, después de mirar cada grupo de forma individual hemos captado que hay un total de 18 grupos de investigación que su página web no sigue esta estructura. A continuación detallamos los enlaces de estos grupos: 

* https://engalim.uib.cat/Grupo-investigador/
* https://gicafe.uib.cat/Estructura_I_Personal/
* https://eic.uib.cat/Equip-investigador/
* https://gedhe.uib.cat/Equip-investigador/
* https://gifes.uib.cat/Research-staff/
* https://meteo.uib.cat/equip/
* https://recumare.uib.cat/EquipInvestigador/
* https://reacmole.uib.cat/Equip-investigador/
* https://ccts.uib.cat/Equipo-investigador/
* https://desigualtats.uib.cat/Equip-investigador/
* https://gresib.uib.cat/Equip-investigador/
* https://relatmit.uib.cat/Equip-investigador/
* https://grupestudidha.uib.cat/Equip-investigador/
* https://praxis.uib.cat/Equip/
* https://imasdel.uib.cat/Equip/
* https://ndpc.uib.cat/Equip-investigador/
* https://acsic.uib.cat/EquipInv/
* https://scopia.uib.cat/Equip/

El primer enlace es un grupo sumamente raro, con lo cual lo quitaremos de nuestros grupos extraños. Guardaremos estos enlaces en una lista para tratarlos más tarde.

In [3]:
grupos_raros = ["https://gicafe.uib.cat/Estructura_I_Personal/",
                       "https://eic.uib.cat/Equip-investigador/", "https://gedhe.uib.cat/Equip-investigador/",
                       "https://gifes.uib.cat/Research-staff/","https://meteo.uib.cat/equip/",
                       "https://recumare.uib.cat/EquipInvestigador/", "https://reacmole.uib.cat/Equip-investigador/",
                       "https://ccts.uib.cat/Equipo-investigador/", "https://desigualtats.uib.cat/Equip-investigador/",
                       "https://gresib.uib.cat/Equip-investigador/", "https://relatmit.uib.cat/Equip-investigador/",
                       "https://grupestudidha.uib.cat/Equip-investigador/", "https://praxis.uib.cat/Equip/",
                       "https://imasdel.uib.cat/Equip/", "https://ndpc.uib.cat/Equip-investigador/",
                       "https://acsic.uib.cat/EquipInv/", "https://scopia.uib.cat/Equip/"]

Al entrar tantas veces por las mismas páginas de la UIB, la página web ha empezado a bloquear la entrada y no podemos scrappear la información de los investigadores. Añadimos el parámetro headers al utilizar la función `requests.get` para que nos detecte como un usuario y no un bot. La función `load_page` lee los enlaces url que le pasamos.

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
           'AppleWebKit/537.36 (KHTML, like Gecko) '\
           'Chrome/75.0.3770.80 Safari/537.36'}
def load_page(url):
    pagina_categorias = requests.get(url, headers = headers)
    if pagina_categorias.status_code == 200:
        return BeautifulSoup(pagina_categorias.text,'html5')
    else:
        print("Algo va mal")
        return None

La función `extrae_grupo` tiene como parámetro los enlaces de los grupos de investigación y nos devuelve el nombre del grupo en cuestión y el enlace a la página del equipo de investigación del grupo.

In [5]:
def extrae_grupo(item):
    nombre = item.text.strip()
    url = item.find("a").get("href")
    return nombre, url

El tema de extraer el currículum vitae de los investigadores es un tema complicado. Nosotros estamos tratando en general con la página de la UIB en catalán, pero hay cv's que no están disponibles en catalán pero sí en castellano o en inglés. Lo que haremos será un recorrido en el siguiente orden: catalán-castellano-inglés. El procedimiento sería:

* Buscar el cv en la página en catalán y si está finalizamos, si no pasamos al segundo punto.
* Buscamos el cv en la página en castellano y si está finalizamos, si no pasamos al último punto.
* Buscamos el cv en la página en inglés y si está finalizamos, si no devolvemos "No hay cv".

In [10]:
def extrae_cv(url):
    pagina_cv = load_page(url)
    if pagina_cv is not None:
        if pagina_cv.find("div", class_="uib_style_nolanguageversion") is not None:
            url_es = url.replace("cat", "es/es")
            pagina_cv_es = load_page(url_es)
            if pagina_cv_es.find("div", class_="uib_style_nolanguageversion") is not None:
                url_eu = url.replace("cat", "eu")
                pagina_cv_eu = load_page(url_eu)
                if pagina_cv_eu.find("div", class_="uib_style_nolanguageversion") is not None:
                    return "No hay cv"
                if pagina_cv_eu.find("div", id="cv_breve") is not None:
                    return pagina_cv_eu.find("div", id="cv_breve").text
                else:
                    return "No hay cv"
            if pagina_cv_es.find("div", id="cv_breve") is not None:
                return pagina_cv_es.find("div", id="cv_breve").text
            else:
                return "No hay cv"
        if pagina_cv.find("div", id="cv_breve") is not None:
            return pagina_cv.find("div", id="cv_breve").text
        else: 
            return "No hay cv"
    else:
        return "No hay cv"

La función principal que utilizaremos para extraer la información de los investigadores es `extra_investigador` con parámetros la url de la página del investigador, el tipo de investigador y el grupo al que pertenece. Utilizaremos dentro de esta función la función `extrae_cv` mencionada anteriormente. 

In [13]:
def extra_investigador(item, tipo, grupo):
    cv = None
    #Si no existe la página del investigador devolvemos None
    if item.find("a") == None:
        url = None  
    #Si existe guardamos en url el enlace
    else:    
        url = item.find("a").get("href") 
    texto = item.text 
    partes = texto.split('(')
    p1 = partes[0].strip().split()
    nombre = " ".join(p1[1:])
    if p1[0][2]=='a': 
        genero = "FEMENINO"
    else:
        genero = "MASCULINO"
    tipo_investigador = tipo
    titulo = p1[0]
    #Puede no tener especificado el puesto del investigador, si es así devolvemos "No se sabe"
    if len(partes) == 2: 
        relacion_uib = partes[1][:-1]
    else: 
        relacion_uib = "No se sabe"
    #Si existe la página del investigador no es None extraemos su cv
    if url is not None: 
        cv = extrae_cv(url)
    else:
        cv = "No hay cv"
    #Devolvemos la información como un diccionario
    return {"grupo" : grupo, "nombre" : nombre, "genero" : genero, "tipo de investigador" : tipo_investigador, 
            "titulo" : titulo, "puesto" : relacion_uib, "cv" : cv} 

Declaramos una lista vacía llamada `investigadores` donde iremos guardando la información de los investigadores.

In [14]:
investigadores = []

Llegamos al cuerpo principal del trabajo. Consiste en una serie de bucles `for` para ir recorriendo las páginas de forma gradual y extraemos la información de los investigadores. La estructura del algoritmo se especifica de la siguiente forma:

* El primer `for` es para ir recorriendo las 15 páginas que contienen los diferentes grupos de investigación que hay.
* El segundo `for` lo utilizamos para entrar en cada grupo de investigación y en la página del equipo para conseguir la lista de los investigadores.
* El tercer `for` sirve para guardar en un la variable `personal` la lista de los investigadores.
* El cuarto y último `for` ya es el último paso, extraemos la información de los investigadores. 

In [15]:
for i in range(1,16):
    #Leemos en cada iteración la página principal donde están los grupos de investigación
    page = load_page("https://www.uib.cat/recerca/estructures/grups/grups_area/id_area=-1%2526npag=" + str(i))
    if page == None:
        continue
    #Guardajmos los grupos de investigación
    contenedor = page.find("div", class_="uib_style_filaunica")
    grupos = contenedor.find_all("li")
    for j in range(0, len(grupos)):
        #Extraemos el nombre del grupo y el enlace a su equipo de investigación
        a, b = extrae_grupo(grupos[j])
        web_grupo = b + "/equip/index.html"
        pgrupo = load_page(web_grupo)
        #Si no existe el enlace y es uno de los 18 casos extraños mencionados anteriormente, seguimos con la siguiente iteración
        if pgrupo == None:
            continue
        #Guardamos la lista de las categorías de investigadores
        contenido = pgrupo.find("div", itemprop="mainContentOfPage")
        lista_rel = contenido.find_all("h3")
        for k in range(0, len(lista_rel)):
            #Guardamos los investigadores de cada categoría que estamos recorriendo
            personal = lista_rel[k].find_next_sibling().find_all("li")
            for l in range(0, len(personal)):
                #Extraemos la información de cada investigador
                investigadores.append(extra_investigador(personal[l], lista_rel[k].text, a))

Algo va mal
Algo va mal
Algo va mal
Algo va mal
Algo va mal
Algo va mal
Algo va mal
Algo va mal
Algo va mal
Algo va mal
Algo va mal
Algo va mal
Algo va mal
Algo va mal
Algo va mal
Algo va mal
Algo va mal
Algo va mal


Una vez que hemos extraído los datos de los investigadores de las páginas regulares es hora de extraerlas de las páginas raras que hemos guardado en la lista `grupos_raros`. Guardamos estos investigadores también en la lista `investigadores`.

In [39]:
for i in range(0, len(grupos_raros)):
        pgrupo = load_page(grupos_raros[i])
        if pgrupo == None:
            continue
        titulo = pgrupo.find("div", id = "menu-navegacio-seccio")
        a = titulo.find("a").text
        contenido = pgrupo.find("div", itemprop="mainContentOfPage")
        lista_rel = contenido.find_all("h3")
        for k in range(0, len(lista_rel)):
            personal = lista_rel[k].find_next_sibling().find_all("li")
            for l in range(0, len(personal)):
                investigadores.append(extra_investigador(personal[l], lista_rel[k].text, a))

Ahora utilizamos la función `dump_data` para grabar los datos en un fichero llamado también `investigadores`.

In [47]:
dump_data(investigadores, "investigadores.json")

El trabajo estaría ya realizado pero si se quiere hacer comprobaciones de los datos obtenidos, siempre se puede cargar el fichero `investigadores` con la función `load_page`. 